In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
%run ../ipynb_load_data_natural.py

{'SDG': Value(dtype='int64', id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None)}
Example instance:	 {'SDG': 8, 'ABSTRACT': 'The scheme gives enterprises with business activity in Norway a tax credit on their R&D projects. The R&D content must be approved by the Research Council of Norway ex ante. In 2009, the cap on expenses per enterprise for intramural R&D projects increased to NOK 5.5 million (previously it was N0K 4 million), and NOK11 million (previously it was NOK 8 million) for projects conducted at an R&D institution.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None, '__index_level_0__': 492}
id2label: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
label2id: {'1': 0, '2': 1, '

# LLaMA3 8B

In [4]:
from transformers import LlamaTokenizer, AutoTokenizer

# base model
HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-{DATASET_TYPE.value}" + ("-lower" if SHOULD_LOWER else "")

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
# print(tokenizer.pad_token, tokenizer.eos_token)

encoded_dataset = dataset.map(preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <enum 'DatasetType'>.
  StockPickler.save(self, obj, save_persistent_id)
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <enum 'DatasetType'>: __main__.DatasetType has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/630 [00:00<?, ? examples/s]

Map:   0%|          | 0/271 [00:00<?, ? examples/s]

## Model

In [5]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model
from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    #num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,885,376 || all params: 7,511,879,680 || trainable%: 0.0917


In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 4
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/final/{MODEL_NAME}",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
    run_name=MODEL_NAME
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.726500,2.527387,0.273063,0.304525,0.273063,0.256401
2,1.715100,1.309559,0.590406,0.679420,0.590406,0.598111
3,0.709700,0.954799,0.723247,0.736581,0.723247,0.716193
4,0.283400,0.903933,0.756458,0.761154,0.756458,0.754213
5,0.069500,1.105640,0.774908,0.791816,0.774908,0.776837
6,0.023300,0.919095,0.789668,0.787923,0.789668,0.786231
7,0.002800,0.988846,0.819188,0.819244,0.819188,0.816442
8,0.000100,0.967157,0.811808,0.812617,0.811808,0.809036
9,0.000100,0.933073,0.811808,0.809941,0.811808,0.807585
10,0.000000,0.972420,0.815498,0.811442,0.815498,0.810718


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/user/dbielik/msc-thesis/venv/lib/python3.1

TrainOutput(global_step=3160, training_loss=0.3265398293262041, metrics={'train_runtime': 5461.3057, 'train_samples_per_second': 2.307, 'train_steps_per_second': 0.579, 'total_flos': 2.704295212351488e+17, 'train_loss': 0.3265398293262041, 'epoch': 20.0})

In [8]:
trainer.evaluate()

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.9832360744476318,
 'eval_accuracy': 0.8265682656826568,
 'eval_precision': 0.8223278978741205,
 'eval_recall': 0.8265682656826568,
 'eval_f1': 0.8216155452410531,
 'eval_runtime': 31.3423,
 'eval_samples_per_second': 8.646,
 'eval_steps_per_second': 2.17,
 'epoch': 20.0}

In [9]:
from sklearn.metrics import classification_report

trainer.model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = trainer.model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))], digits=4)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [10]:
print(report)

              precision    recall  f1-score   support

       SDG 1     0.7778    0.8235    0.8000        17
       SDG 2     0.8824    0.8824    0.8824        17
       SDG 3     0.7500    0.8824    0.8108        17
       SDG 4     1.0000    1.0000    1.0000        17
       SDG 5     0.7647    0.7647    0.7647        17
       SDG 6     0.8889    1.0000    0.9412        16
       SDG 7     0.8000    0.7500    0.7742        16
       SDG 8     0.7368    0.8235    0.7778        17
       SDG 9     0.7222    0.7647    0.7429        17
      SDG 10     0.5385    0.4118    0.4667        17
      SDG 11     0.8421    0.9412    0.8889        17
      SDG 12     0.9167    0.6471    0.7586        17
      SDG 13     0.8125    0.7647    0.7879        17
      SDG 14     1.0000    1.0000    1.0000        17
      SDG 15     0.8000    0.7059    0.7500        17
      SDG 16     0.6842    0.7647    0.7222        17
      SDG 17     0.0000    0.0000    0.0000         1

    accuracy              

In [18]:
sample_sentence = """Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, and halt and reverse land degradation and halt biodiversity loss """
probs = torch.nn.functional.softmax(model(**tokenizer(sample_sentence, return_tensors="pt", truncation=True, max_length=512).to(model.device)).logits, dim=-1)

for i, prob in enumerate(probs[0]):
    print(f"Label {model.config.id2label[i]}: {prob.item()}")

print(model.config.id2label)
print(model.config.label2id)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Label 1: 2.4199485778808594e-05
Label 2: 0.033447265625
Label 3: 0.0024566650390625
Label 4: 0.0002231597900390625
Label 5: 0.00015544891357421875
Label 6: 0.0054931640625
Label 7: 0.0830078125
Label 8: 0.010986328125
Label 9: 0.014404296875
Label 10: 0.5
Label 11: 0.00439453125
Label 12: 0.00170135498046875
Label 13: 0.01318359375
Label 14: 0.06396484375
Label 15: 0.259765625
Label 16: 0.002899169921875
Label 17: 0.0022430419921875
{0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17'}
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9, '11': 10, '12': 11, '13': 12, '14': 13, '15': 14, '16': 15, '17': 16}
